<a href="https://colab.research.google.com/github/udeigwe/rnn_tutorial/blob/main/rnn_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem: Determining Whether a Given Text String is Positive or Negative.**

We’ll implement an RNN from scratch to perform a simple Sentiment Analysis task: determining whether a given text string is positive or negative.

Adapted from https://victorzhou.com/blog/intro-to-rnns/



# Data Pre-processing

### The dataset consists of two Python dictionaries:

In [61]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}


### Create the vocabulary

In [62]:
#Create the vocabulary.
#vocab = list(w for text in train_data.keys())

vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print('%d unique words found' %vocab_size) # 18 unique words found

18 unique words found


### Assign indices to each word.

In [63]:
# Assign indices to each word.
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }
print(word_to_idx['good']) # 16 (this may change)
print(idx_to_word[0]) # sad (this may change)

9
happy


In [64]:
word_to_idx

{'all': 17,
 'am': 12,
 'and': 11,
 'at': 14,
 'bad': 8,
 'earlier': 13,
 'good': 9,
 'happy': 0,
 'i': 10,
 'is': 3,
 'not': 2,
 'now': 5,
 'or': 7,
 'right': 1,
 'sad': 16,
 'this': 4,
 'very': 6,
 'was': 15}

### Converting words to one-hot vector
The “one” in each one-hot vector will be at the word’s corresponding integer index.



In [65]:
# We’ll use one-hot vectors, which contain all zeros except for a single one. 
# The “one” in each one-hot vector will be at the word’s corresponding integer index.

import numpy as np

def createInputs(text):
  '''
  Returns an array of one-hot vectors representing the words
  in the input text string.
  - text is a string
  - Each one-hot vector has shape (vocab_size, 1)
  '''
  inputs = []
  for w in text.split(' '):
    v = np.zeros((vocab_size, 1))
    v[word_to_idx[w]] = 1
    inputs.append(v)
  return inputs

# RNN: The Forward Phase
It’s time to start implementing our RNN! We’ll start by initializing the 3 weights and 2 biases our RNN needs:

In [66]:
import numpy as np
from numpy.random import randn

class RNN:
  # A Vanilla Recurrent Neural Network.

  def __init__(self, input_size, output_size, hidden_size=64):
    # Weights
    self.Whh = randn(hidden_size, hidden_size) / 1000
    self.Wxh = randn(hidden_size, input_size) / 1000
    self.Why = randn(output_size, hidden_size) / 1000

    # Biases
    self.bh = np.zeros((hidden_size, 1))
    self.by = np.zeros((output_size, 1))

  def forward(self, inputs):
    '''
    Perform a forward pass of the RNN using the given inputs.
    Returns the final output and hidden state.
    - inputs is an array of one-hot vectors with shape (input_size, 1).
    '''
    h = np.zeros((self.Whh.shape[0], 1)) 
    # a column vector, the same size as number of hidden units?

    self.last_inputs = inputs
    self.last_hs = { 0: h }

    # Perform each step of the RNN
    for i, x in enumerate(inputs):
      h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
      self.last_hs[i + 1] = h

    # Compute the output
    y = self.Why @ h + self.by

    return y, h

  def backprop(self, d_y, learn_rate=2e-2):
    '''
    Perform a backward pass of the RNN.
    - d_y (dL/dy) has shape (output_size, 1).
    - learn_rate is a float.
    '''
    n = len(self.last_inputs)

    # Calculate dL/dWhy and dL/dby.
    d_Why = d_y @ self.last_hs[n].T
    d_by = d_y


In [82]:
def softmax(xs):
  # Applies the Softmax Function to the input array.
  return np.exp(xs) / sum(np.exp(xs))

# Initialize our RNN!
rnn = RNN(vocab_size,2)

# Try running without training
inputs = createInputs('i am very good')
out, h = rnn.forward(inputs)
probs = softmax(out)
#print(probs) # [[0.50000095], [0.49999905]]

count=0
# Loop over each training example
for x, y in train_data.items():
  inputs = createInputs(x) 
      #imputs is an array of vectors,
      # each vector is a one-hot vector
      # respresenting a word in an input sentence 
  target = int(y)
      # target is 0 or 1 
      # for negative or positive

  # Forward
  out, _ = rnn.forward(inputs)
  probs = softmax(out)

  # Build dL/dy
  d_L_d_y = probs 
      # d_L_d_y is a two element vector
      # of the probability of output being 
      # 0 or 1
  d_L_d_y[target] -= 1
  print(d_L_d_y[target])

  #Backward
 # rnn.backprop(d_L_d_y)
 
   



[-0.50000446]
[-0.50000158]
[-0.5000048]
[-0.50000414]
[-0.49999548]
[-0.49999847]
[-0.49999515]
[-0.49999591]
[-0.50000445]
[-0.50000159]
[-0.50000479]
[-0.50000415]
[-0.5000048]
[-0.5000045]
[-0.50000158]
[-0.50000154]
[-0.50000414]
[-0.50000411]
[-0.49999515]
[-0.49999548]
[-0.49999847]
[-0.49999591]
[-0.50000479]
[-0.50000445]
[-0.50000159]
[-0.50000415]
[-0.50000479]
[-0.49999847]
[-0.49999847]
[-0.49999548]
[-0.5000048]
[-0.49999515]
[-0.49999549]
[-0.49999846]
[-0.49999516]
[-0.4999959]
[-0.49999516]
[-0.49999896]
[-0.50000104]
[-0.50000104]
[-0.50000104]
[-0.50000276]
[-0.50000278]
[-0.49999726]
[-0.49999727]
[-0.50000104]
[-0.49999896]
[-0.50000104]
[-0.49999726]
[-0.50000276]
[-0.49999726]
[-0.50000278]
[-0.49999727]
[-0.50000274]
[-0.49999722]
[-0.49999896]
[-0.50000104]
[-0.49999724]


In [68]:
inputs;

In [69]:
A={1:10, 5:20, 7:30}

In [70]:
A

{1: 10, 5: 20, 7: 30}

In [71]:
A[1]

10

In [72]:
int(True)

1

In [73]:
int(False)

0

In [74]:
A[1] -= 1

In [75]:
A[1]

9

In [76]:
print(len(A))

3


In [77]:
A = [[1,2,3], [3, 1, 0]]

In [78]:
v=[[1], [1]]

In [79]:
A.append(v)

In [80]:
print(A)

[[1, 2, 3], [3, 1, 0], [[1], [1]]]
